# Classification 10/20/22

Goal: Start classifying tidydata_1020222 using sktime module, specificaly classify based on differences in temp / rh 
Update 11/10/22: introduce functions, start classification with sklearn bc clearer error messages and better documentation 



In [133]:
import json
import datetime
import pandas as pd
import numpy as np
import sys
import os
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import plotly.graph_objects as go

In [2]:
df = pd.read_csv("../constants/td_ambient_102022.csv", )
df

,DateTime,Temp C,RH %,Room,Ambient Temp,Ambient RH,Window Open
0,2022-07-20 07:20:00,23.189,51.268,0,14.333333,30.555556,1
1,2022-07-20 07:20:30,23.189,51.362,0,14.333333,30.555556,1
2,2022-07-20 07:21:00,23.189,51.488,0,14.333333,30.555556,1
3,2022-07-20 07:21:30,23.189,51.457,0,14.333333,30.555556,1
4,2022-07-20 07:22:00,23.165,51.517,0,14.333333,30.555556,1
...,...,...,...,...,...,...,...
123601,2022-09-11 07:58:00,29.196,50.861,1,19.611111,30.000000,0
123602,2022-09-11 07:58:30,29.196,50.861,1,19.611111,30.000000,0
123603,2022-09-11 07:59:00,29.196,50.796,1,19.611111,30.000000,0
123604,2022-09-11 07:59:30,29.196,50.796,1,19.611111,30.000000,0


In [3]:
# add temperature and rh deltas for each observation 
df["T_Delta"] =  df["Temp C"] - df["Ambient Temp"]
df["RH_Delta"] =  df["RH %"]- df["Ambient RH"]


In [4]:
df.head()

,DateTime,Temp C,RH %,Room,Ambient Temp,Ambient RH,Window Open,T_Delta,RH_Delta
0,2022-07-20 07:20:00,23.189,51.268,0,14.333333,30.555556,1,8.855667,20.712444
1,2022-07-20 07:20:30,23.189,51.362,0,14.333333,30.555556,1,8.855667,20.806444
2,2022-07-20 07:21:00,23.189,51.488,0,14.333333,30.555556,1,8.855667,20.932444
3,2022-07-20 07:21:30,23.189,51.457,0,14.333333,30.555556,1,8.855667,20.901444
4,2022-07-20 07:22:00,23.165,51.517,0,14.333333,30.555556,1,8.831667,20.961444


# Grouping and Statistics 

In [5]:
# do some grouping 

# group by window open vs window closed -> where are the deltas greater 
df.drop("DateTime", axis=1).groupby(["Window Open"]).agg(["mean", "std"]).loc[:,["T_Delta", "RH_Delta"]]

T_Delta             RH_Delta          
                 mean       std       mean       std
Window Open                                         
0            5.752171  4.206669  30.640394  8.547424
1            6.257008  3.594757  28.279236  7.846737

In [6]:
# group by rooms -> where are the deltas greater 
df.drop("DateTime", axis=1).groupby(["Room"]).agg(["mean", "std"]).loc[:,["T_Delta", "RH_Delta"]]

T_Delta             RH_Delta          
          mean       std       mean       std
Room                                         
0     5.729435  3.780207  29.178837  8.143704
1     6.301897  4.010265  29.637181  8.395411

In [7]:
# group by window open vs window closed and rooms -> where are the deltas greater 
df.drop("DateTime", axis=1).groupby(["Window Open", "Room"]).agg(["mean", "std"]).loc[:,["T_Delta", "RH_Delta"]]

T_Delta             RH_Delta          
                      mean       std       mean       std
Window Open Room                                         
0           0     1.603624  4.786734  37.193811  7.443892
            1     6.059046  3.992906  30.155626  8.423356
1           0     6.020292  3.521054  28.613806  7.889349
            1     8.272099  3.584829  25.431145  6.841461

# Group data at various time intervals 

## hourly 

In [8]:
times = pd.to_datetime(df.DateTime)

In [9]:
# aggregate data based on hour and room 
df_hour = df.groupby([times.dt.date, times.dt.hour, "Room"]).mean()
df_hour.head()

Temp C       RH %  Ambient Temp  Ambient RH  \
DateTime   DateTime Room                                                   
2022-07-20 7        0     23.326400  51.622238     14.333333   30.555556   
                    1     23.422100  52.867775     14.333333   30.555556   
           8        0     23.736167  50.783575     14.666667   30.555556   
                    1     23.740725  52.640442     14.666667   30.555556   
           9        0     24.307792  50.038683     15.388889   28.888889   

                          Window Open   T_Delta   RH_Delta  
DateTime   DateTime Room                                    
2022-07-20 7        0          1.0000  8.993067  21.066682  
                    1          0.2375  9.088767  22.312219  
           8        0          1.0000  9.069500  20.228019  
                    1          1.0000  9.074058  22.084886  
           9        0          1.0000  8.918903  21.149794

## half-hourly

In [157]:
def make_df_with_freq(df, freq):
    dftime = df.copy()

    # transform datetime index to actual pandas datetime, and set as index of dataframe 
    dftime["DateTime"] = pd.to_datetime(dftime["DateTime"])
    dftime.set_index("DateTime", inplace=True)

    # resample the dataframe to group by the correct index and drop nans
    dftime_freq = dftime.groupby("Room").resample(freq).mean()
    dftime_freq.dropna(inplace=True)

    # map window variables to 0 or 1
    dftime_freq["Window Open"] = dftime_freq["Window Open"].round()

    return dftime_freq


In [158]:
df_30min = make_df_with_freq(df, "30T")
df_15min = make_df_with_freq(df, "15T")
df_15min = make_df_with_freq(df, "15T")
df_30min

Temp C       RH %  Room  Ambient Temp  \
Room DateTime                                                        
0    2022-07-20 07:00:00  23.180600  51.421100   0.0     14.333333   
     2022-07-20 07:30:00  23.375000  51.689283   0.0     14.333333   
     2022-07-20 08:00:00  23.581400  51.084233   0.0     14.666667   
     2022-07-20 08:30:00  23.890933  50.482917   0.0     14.666667   
     2022-07-20 09:00:00  24.194967  50.202333   0.0     15.388889   
...                             ...        ...   ...           ...   
1    2022-09-11 06:00:00  29.271000  50.757817   1.0     19.500000   
     2022-09-11 06:30:00  29.255583  50.776800   1.0     19.500000   
     2022-09-11 07:00:00  29.241833  50.811517   1.0     19.611111   
     2022-09-11 07:30:00  29.219333  50.808100   1.0     19.611111   
     2022-09-11 08:00:00  29.196000  50.796000   1.0     19.833333   

                          Ambient RH  Window Open   T_Delta   RH_Delta  
Room DateTime                                                           
0    2022-07-20 07:00:00   30.555556          1.0  8.847267  20.865544  
     2022-07-20 07:30:00   30.555556          1.0  9.041667  21.133728  
     2022-07-20 08:00:00   30.555556          1.0  8.914733  20.528678  
     2022-07-20 08:30:00   30.555556          1.0  9.224267  19.927361  
     2022-07-20 09:00:00   28.888889          1.0  8.806078  21.313444  
...                              ...          ...       ...        ...  
1    2022-09-11 06:00:00   30.000000          0.0  9.771000  20.757817  
     2022-09-11 06:30:00   30.000000          0.0  9.755583  20.776800  
     2022-09-11 07:00:00   30.000000          0.0  9.630722  20.811517  
     2022-09-11 07:30:00   30.000000          0.0  9.608222  20.808100  
     2022-09-11 08:00:00   30.000000          0.0  9.362667  20.796000  

[2066 rows x 8 columns]

In [160]:
freqs = ["60T", "30T", "15T", "10T", "5T", "1T", "30s"]
df_freq_dict = {}
for freq in freqs:
    df_freq_dict[freq] = make_df_with_freq(df, freq)

## create data for classification 

In [175]:
def make_univ_classif_data(df, x_var, y_var, seed=42):
    """make univariate classification data, split data into train and test and assert they have the right shapes.. """
    x = df.loc[:, [x_var]]
    y = df.loc[:, y_var]

    # convert from pandas series to numpy and reshape 
    xnp = x.to_numpy().flatten().reshape(-1, 1)
    ynp = y.to_numpy()
    # turn into categorical variable TODO -> better with np masks.... 
    ynp = np.array([0 if i < 0.5 else 1 for i in ynp ])

    # assert that shapes are as needed for processing 
    assert(xnp.shape == (len(xnp),1))
    assert(ynp.shape == (len(ynp),))

    # split into test and training groups 
    x_train, x_test, y_train, y_test = train_test_split(xnp, ynp,test_size=0.2, random_state=seed)

    data_dict = {
        "x_train": x_train,
        "x_test": x_test,
        "y_train": y_train,
        "y_test": y_test
    }

    return data_dict
    

### sklearn logistic regression 

In [69]:
def run_log_reg(data):
    " runs regression and note performance, taking in dictionary of required data "
    # TODO -> assertions that data is in correct format 
    logReg = LogisticRegression()
    logReg.fit(data["x_train"], data["y_train"])
    score = logReg.score(data["x_test"], data["y_test"])
    return score 

In [78]:
features = ['Temp C', 'RH %', 'Ambient Temp', 'Ambient RH', 'T_Delta', 'RH_Delta']

feature_scores = {}

for feature in features:
    d = make_univ_classif_data(df_hour, feature, "Window Open")
    score = run_log_reg(d)
    feature_scores[feature] = np.round(score,3)

# pd.DataFrame(feature_scores, index=["scores"]).T

{'Temp C': 0.668,
 'RH %': 0.611,
 'Ambient Temp': 0.572,
 'Ambient RH': 0.567,
 'T_Delta': 0.553,
 'RH_Delta': 0.562}

In [173]:
def see_scores(seed=42):
    df_freq_scores = {}
    for k,v in df_freq_dict.items():
        feature_scores = {}
        for feature in features:
            d = make_univ_classif_data(v, feature, "Window Open", seed)
            score = run_log_reg(d)
            feature_scores[feature] = np.round(score,3)
        df_freq_scores[k] = feature_scores

    return pd.DataFrame(df_freq_scores)

In [176]:
see_scores(seed=42)

,60T,30T,15T,10T,5T,1T,30s
Temp C,0.582,0.626,0.642,0.624,0.619,0.601,0.620
RH %,0.505,0.534,0.562,0.542,0.536,0.520,0.537
Ambient Temp,0.591,0.524,0.568,0.544,0.565,0.541,0.552
Ambient RH,0.567,0.519,0.556,0.547,0.552,0.532,0.541
T_Delta,0.553,0.531,0.562,0.541,0.550,0.532,0.540
RH_Delta,0.601,0.531,0.561,0.524,0.544,0.533,0.542


In [177]:
see_scores(seed=4)

,60T,30T,15T,10T,5T,1T,30s
Temp C,0.630,0.657,0.581,0.611,0.597,0.616,0.614
RH %,0.514,0.536,0.525,0.513,0.520,0.525,0.530
Ambient Temp,0.558,0.539,0.539,0.557,0.546,0.555,0.558
Ambient RH,0.548,0.522,0.527,0.545,0.534,0.548,0.544
T_Delta,0.524,0.531,0.533,0.550,0.540,0.544,0.546
RH_Delta,0.534,0.517,0.540,0.548,0.537,0.545,0.542
